In [1]:
#Import required packages

import brightway2 as bw
import numpy as np
import pandas as pd
from bw2data.parameters import ProjectParameter
import sys
import matplotlib.pyplot as plt
from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter, Group


#Import the battery design module from a local path
sys.path.insert(1, r'C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\ME_battery_supply_chain\Model')
import battery_design.battery_design as bd


import battery_emissions as bat_lca
import battery_cost as bat_cost


path_batpac = r"C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\Case study\BatPaC 4.0 - 1Oct2020 - Original.xlsm"


## Calculate LCIA module values

In [2]:
#Brightway2 general setup:
bw.projects.set_current('parameterised_battery_lca')
bw.bw2setup()
#Import ecoinvent 3.7.1:
if 'eidb 3.7' in bw.databases:
    pass
else:
    pathi37 = 'path to ecoinvent'
    eidb37 = bw.SingleOutputEcospold2Importer(pathi37, 'eidb 3.7')
    eidb37.apply_strategies()
    eidb37.statistics()
    
#Battery LCA setup:
# Import all Brightway2 databases (BW2Package format) required for the battery LCA: 
bat_lca.import_db_brightway()

# Import the default project parameters such as process yield and energy consumption (see Excel file: 'bw_default_project_parameters.xlsx' ):
bat_lca.import_project_parameters()

# # Import the BW activity functions CHECK IF NEEDED IF THE BW2PACKAGES ARE UPLOADED!!!!

Biosphere database already present!!! No setup is needed
anode_material  already present
anode_materials  already present
battery_production  already present
cathode  already present
cathode_material  already present
cell_container  already present
cell_other  already present
current_collectors  already present
cut_off_materials  already present
cut_off_materials  already present
cut_off_production  already present
electrolyte  already present
manufacturing_waste_scrap  already present
module  already present
MyModelName  already present
other_material  already present
other_materials  already present
pack  already present
separator  already present
separators  already present


## Solve battery pack design

In [3]:
## Solve single pack design in BatPaC for base case:

#Establish base case pack:
battery = bd.Battery_system ( vehicle_type='EV', 
                             electrode_pair='NMC622-G',  
                             silicon_anode= 0.00, 
                             pack_energy=40,)
#Establish parameter dictionary
parameter_dict = battery.parameter_dictionary()

#Solve battery design in BatPaC:
battery_design = bd.solve_batpac_battery_system (batpac_path = path_batpac, parameter_dict=parameter_dict, visible=False)

## Calculate modular emissions base case:

In [4]:
#Include default process parameters:
process_parameters = {
    'py_am_mixing': 0.99,
    'py_cell_aging': 0.95,
    'py_electrode_stacking':0.99,
    'py_electrolyte_filling':0.94,
    'py_foil_coating': 0.99,
    'py_foil_slitting': 0.92,
    'py_nmp_recovery': 0.995,
    'py_separator_stacking':0.98,
    'py_slurry_coating': 0.95,
    'py_slurry_slitting':0.99,
    'battery_manufacturing_capacity': 100000, #Total packs per year
    'battery_manufacturing_location': 'RER'  # Battery production
}

#Dictionary of all system parameters (non-alphanumeric parameters) needed to solve LCA in Brightway. Parameters match Brightway project parameters:
project_param_all = bat_lca.parameter_dictionary (battery_design['material_content_pack'], process_parameters, battery_design['general_battery_parameters'])


In [5]:
#Update Brightway default values with battery design values:
bat_lca.update_param_battery_bw(project_param_all)

In [6]:
#Check if parameter matches Brightway project parameters:

bw_project_names = [x.name for x in ProjectParameter.select()]
for param in project_param_all.keys():
    if param in bw_project_names:
        select = [x for x in ProjectParameter.select() if x.name == param][0]
        if project_param_all[param] != select.amount:
            print (f"Parameter {param} with value {project_param_all[param]} is not equal to Brightway project parameter {select.amount}")
            
        



In [7]:
# Cut off the exchanges if activity if present in the 'cut off database'. 
# E.g. 6um copper foil production is present in the 'cut of database' and will be cut in the anode coating process. All cut-off exchanges are set to zero
cut_off_dict = bat_lca.modules_with_cuts('cut_off_materials')

# # Divide all battery production activities by total battery system weight:
bat_product_act = [act['name'] for act in bw.Database('battery_production')]

for act in bat_product_act:
    pack_weight = project_param_all['battery_pack']

    cut_off_dict[act]['amount'] = cut_off_dict[act]['amount']/pack_weight
    for exc in cut_off_dict[act]['cuts'].keys():
        cut_off_dict[act]['cuts'][exc][1] = cut_off_dict[act]['cuts'][exc][1]/pack_weight

# #Cut-off exchanges:
bat_lca.cut_modules_to_zero(cut_off_dict)

# # Establish product-module dataframe of base system
technology_matrix_cut_off = bat_lca.modular_technology_matrix(cut_off_dict)        

# #Select all Recipe Midpoint impacts:
impacts = [m for m in bw.methods if 'ReCiPe Midpoint (H) V1.13' in str(m) and 'no LT' in str(m)]

#Establish numpy array of module emissions:
H_m = bat_lca.lcia_modules (cut_off_dict, impacts)

## Total emissions pack:



In [8]:
A_matrix = technology_matrix_cut_off
#Parameter shortcuts:
pack_weight = battery_design['material_content_pack']['battery pack']
capacity = battery_design['general_battery_parameters']['battery_capacity']

#Inverse the A' matrix:
A_inv = pd.DataFrame(np.linalg.pinv(A_matrix.values), A_matrix.columns, A_matrix.index)

#Establish final product demand vector for 1 battery based on pack weight
y_prime = pd.Series(data = 0, index=A_matrix.index)
y_prime.loc['battery pack'] = pack_weight

#Calculate scaling vector:
s_prime = A_inv.dot(y_prime)

# Emissions for all categories and modules:
h_all =  np.multiply(np.transpose([s_prime]),H_m)

## Export base to Excel


In [109]:
df_h=pd.DataFrame(H_m,A_matrix.columns, columns=impacts).to_excel('H_all.xlsx')

In [9]:

#Remove duplicate products on A rows:

A_matrix_rect = A_matrix[~A_matrix.index.duplicated(keep='first')]
#Save to excel
A_matrix_rect.to_excel('A_base.xlsx')

In [35]:
123/62

1.9838709677419355

In [31]:
123/265

0.4641509433962264